In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
import plotly.graph_objects as graph_objects
import findspark

findspark.init()

# spark_context = SparkContext.getOrCreate()
spark_context = SparkContext('local', 'R1_A1')
spark_session = SparkSession(spark_context)
sqlContext = SQLContext(spark_context)

/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
text_file = sqlContext.read.format('com.databricks.spark.csv').\
    options(header='true', inferschema='true', quote='"', delimiter=',').\
    load('./work/data/GuatemalaExportsTo.csv')

rddfiltro = text_file.rdd.map(tuple)
rddGen = rddfiltro.map(lambda word: (word[4], word[1]))
# rddGen.take(10)

In [8]:
rddConteo = rddGen.reduceByKey(lambda a, b: a+b)
print(f'Conteo total -> {rddConteo.collect()}')

Conteo total -> [('China', 64176766.0), ('Mexico', 511215129.0), ('Canada', 222575942.0), ('Japon', 164579976.0), ('United Kingdom', 101910872.0), ('Costa Rica', 428419775.0), ('Saudi Arabia', 111274535.0), ('Gabon', 6420.0), ('Italy', 166680831.0), ('France', 34023574.0), ('Brazil', 39146101.0), ('Germany', 144422819.0), ('Honduras', 967311616.0), ('Venezuela', 14998293.0), ('El Salvador', 1217138626.0), ('United States', 3803123168.0)]


In [11]:
rddOrden = spark_context.parallelize(
    rddConteo.sortBy(lambda a: a[1], False).take(1))
print(f'País con el mayor valor por exportaciones -> {rddOrden.collect()}')

País con el mayor valor por exportaciones -> [('United States', 3803123168.0)]


In [12]:
rddNombres = rddOrden.map(lambda x: (x[0]))
print(rddNombres.collect())

rddTotales = rddOrden.map(lambda x: (x[1]))
print(rddTotales.collect())

['United States']
[3803123168.0]


In [16]:
graph = graph_objects.Figure(
    data=graph_objects.Bar(
        x=rddNombres.collect(),
        y=rddTotales.collect()
    ))

graph.update_layout(
    title_text='País con el mayor valor por exportaciones',
    title_font_size=30,
    yaxis=dict(title='No. Exportaciones', title_font_size=25),
    xaxis=dict(title='País', title_font_size=25))

graph.update_traces(overwrite=True, marker={"opacity": 0.5})
graph.write_html('./work/reports/R1_A1.html', auto_open=True)